# Dictionaries and Sets

Implementations of specialized mappings often extend dict or collections.User
Dict, instead of these ABCs. The main value of the ABCs is documenting and formal‐
izing the minimal interfaces for mappings, and serving as criteria for isinstance tests
in code that needs to support mappings in a broad sense:

In [1]:
from collections import abc
my_dict = {}
isinstance(my_dict, abc.Mapping)

True

All mapping types in the standard library use the basic dict in their implementation,
so they share the limitation that the keys must be hashable (the values need not be
hashable, only the keys)

- An object is hashable if it has a hash value which never changes during its lifetime (it
needs a __hash__() method), and can be compared to other objects (it needs an
__eq__() method). Hashable objects which compare equal must have the same hash
value. […]

In [2]:
tt = (1,2, (30, 40))
hash(tt)

-3907003130834322577

Given these ground rules, you can build dictionaries in several ways.

In [3]:
a = dict(one=1, two=2, three=3)
b = {'one': 1, 'two': 2, 'three': 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e

True

# Dict Comprehensions

In [4]:
DIAL_CODES = [ 
(86, 'China'),
(91, 'India'),
(1, 'United States'),
(62, 'Indonesia'),
(55, 'Brazil'),
(92, 'Pakistan'),
(880, 'Bangladesh'),
(234, 'Nigeria'),
(7, 'Russia'),
(81, 'Japan'),
]
country_code = {country: code for code, country in DIAL_CODES} 
country_code

{'China': 86,
 'India': 91,
 'United States': 1,
 'Indonesia': 62,
 'Brazil': 55,
 'Pakistan': 92,
 'Bangladesh': 880,
 'Nigeria': 234,
 'Russia': 7,
 'Japan': 81}

In [5]:
{code: country.upper() for country, code in country_code.items() if code < 66}

{1: 'UNITED STATES', 62: 'INDONESIA', 55: 'BRAZIL', 7: 'RUSSIA'}

1. A list of pairs can be used directly with the dict constructor.
2. Here the pairs are reversed: country is the key, and code is the value.
3. Reversing the pairs again, values uppercased and items filtered by code < 66

# Overview of Common Mapping Methods

The basic API for mappings is quite rich. Table 3-1 shows the methods implemented by dict and two of its most useful variations: defaultdict and OrderedDict, both defined in the collections module


The way update handles its first argument m is a prime example of duck typing: it first
checks whether m has a keys method and, if it does, assumes it is a mapping. Otherwise,
update falls back to iterating over m, assuming its items are (key, value) pairs. The
constructor for most Python mappings uses the logic of update internally, which means
they can be initialized from other mappings or from any iterable object producing (key,
value) pairs

## Handling Missing Keys with setdefault

In line with the fail-fast philosophy, dict access with d[k] raises an error when k is not an existing key. Every Pythonista knows that d.get(k, default) is an alternative to d[k] whenever a default value is more convenient than handling KeyError. However, when updating the value found (if it is mutable), using either `__getitem__` or get is
awkward and inefficient. 

setdefault(key, def_value) works in a similar way as get(), but the difference is that each time a key is absent, a new key is created with the def_value associated to the key passed in arguments.


In [6]:
country_code = {'India' : '0091',
				'Australia' : '0025',
				'Nepal' : '00977'}

# Set a default value for Japan
country_code.setdefault('Japan', 'Not Present')

# search dictionary for country code of India
print(country_code['India'])

# search dictionary for country code of Japan
print(country_code['Japan'])

country_code

0091
Not Present


{'India': '0091',
 'Australia': '0025',
 'Nepal': '00977',
 'Japan': 'Not Present'}

# Mapping with Flexible Key Lookup

Sometimes it is convenient to have mappings that return some made-up value when a
missing key is searched. There are two main approaches to this: one is to use a default
dict instead of a plain dict. The other is to subclass dict or any other mapping type
and add a __missing__ method. Both solutions are covered next

## defaultdict: Another Take on Missing Keys

A defaultdict is configured to create items on demand
whenever a missing key is searched. Here is how it works: when instantiating a defaultdict, you provide a callable that is used to produce a default value whenever `__getitem__` is passed a nonexistent key argument

In [7]:
# Python code to demonstrate defaultdict

# importing "collections" for defaultdict
import collections

# declaring defaultdict
# sets default value 'Key Not found' to absent keys
defd = collections.defaultdict(list)

# initializing values
defd['a'] = 1

# initializing values
defd['b'] = 2

# printing value
print ("The value associated with 'a' is : ",end="")
print (defd['a'])

# printing value associated with 'c'
print ("The value associated with 'c' is : ",end="")
print (defd['c'])

The value associated with 'a' is : 1
The value associated with 'c' is : []


## The `__missing__` Method

Underlying the way mappings deal with missing keys is the aptly named `__missing__` method. This method is not defined in the base dict class, but dict is aware of it: if you subclass dict and provide a `__missing__` method, the standard dict.__getitem__will call it whenever a key is not found, instead of raising KeyError


In [8]:
class StrKeyDict0(dict): 
    def __missing__(self, key):
        if isinstance(key, str): 
            raise KeyError(key)
        return self[str(key)] 
    
    def get(self, key, default=None):
        try:
            return self[key] 
        except KeyError:
            return default
         
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys() 

1. StrKeyDict0 inherits from dict.
2. Check whether key is already a str. If it is, and it’s missing, raise KeyError.
3. Build str from key and look it up.
4. The get method delegatesto `__getitem__` by using the self[key] notation; that
gives the opportunity for our `__missing__` to act.
5. If a KeyError was raised, `__missing__` already failed, so we return the default.
6. Search for unmodified key (the instance may contain non-str keys), then for a
str built from the key

In [9]:
d = StrKeyDict0([('2', 'two'), ('4', 'four')])
d['2']

'two'

In [10]:
d[1]

KeyError: '1'

In [11]:
d.get('1', 'Missing')

'Missing'

In [12]:
d.get('2')

'two'

In [13]:
2 in d

True

# Variations of dict

In this section, we summarize the various mapping types included in the collections module of the standard library, besides defaultdict


collections.OrderedDict
- Maintains keys in insertion order, allowing iteration over items in a predictable order. The popitem method of an OrderDict pops the first item by defaly, but if called as my_odict.popitem(last=True), it pops the last item added.

collections.ChainMap
- Holds a list of mappings that can be searched as one. The lookup is performed on each mapping in order, and succeeds if the key is found in any of them. This is useful to interpreters for languages with nested scopes, where each mapping represents a scope context.

collections.Counter
- A mapping that holds an integer count for each key. Updating an existing key adds to its count. This can be used to count instances of hashable objects (the keys) or as a multiset - a set that can hold several occurrences of each element. Counter implements the * and - operators to combine tallies, and other useful emthods such as most_common([n]), which returns an ordered list of tuples witht he n most commmon items and their counts.

collections.UserDict
- A pure python implementation of a mapping that works like a standard dict. Designed to be subclassed

# Subclassing UserDict

The main reason why it’s preferable to subclass from UserDict rather than from dict is that the built-in has some implementation shortcuts that end up forcing us to overrid methods that we can just inherit from UserDict with no problems

Note that UserDict does not inherit from dict, but has an internal dict instance, called data, which holds the actual items. This avoids undesired recursion when coding special methods like `__setitem__`, and simplifies the coding of `__contains__`,

In [14]:
from collections import UserDict

class StrKeyDict(UserDict):
    
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key: object) -> bool:
        return str(key) in self.data
    
    def __setitem__(self, key, item) -> None:
        self.data[str(key)] = item 

1. StrKeyDict extends UserDict.
2. `__missing__` is exactly as in Example 3-7.
3. `__contains__` is simpler: we can assume allstored keys are str and we can check on self.data instead of invoking self.keys() as we did in StrKeyDict0
4. `__setitem__` converts any key to a str. This method is easier to overwrite when we can delegate to the self.data attribute

Because UserDict subclasses MutableMapping, the remaining methods that make StrKeyDict a full-fledged mapping are inherited from UserDict, MutableMapping, or Mapping. The latter have several useful concrete methods, in spite of being abstract base classes (ABCs)


MutableMapping.update
- This powerful method can be called directly but is also used by `__init__` to load the instance from other mappings, from iterables of (key, value) pairs, and key-word arguements. Because it uses self[key] = value to add items, it ends up calling our implentation of `__setitem__`


Mapping.get
- In StrKeyDict0 (Example 3-7), we had to code our own get to obtain results con‐sistent with __getitem__, but in Example 3-8 we inherited Mapping.get, which isimplemented exactly like StrKeyDict0.get

# Immutable Mappings

Since Python 3.3, the types module provides a wrapper class called MappingProxy Type, which, given a mapping, returns a mappingproxy instance that is a read-only but dynamic view of the original mapping. This means that updates to the original mapping can be seen in the mappingproxy, but changes cannot be made through it. 

In [15]:
from types import MappingProxyType

d = {1: 'A'}

d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({1: 'A'})

In [16]:
d_proxy[1]

'A'

In [17]:
d_proxy[2] = 'x'

TypeError: 'mappingproxy' object does not support item assignment

# Set Theory

A set is a collection of unique objects. A basice use case is removing duplicates.

In [18]:
l = ['spam', 'spam', 'eggs', 'spam']
set(l)

{'eggs', 'spam'}

In [19]:
list(set(l))

['eggs', 'spam']

Set elements must be hashable. The set type is not hashable but frozenset is, so you can have frozenset elements inside a set.

In addition to guaranteeing uniqueness, the set types implement the essential set operations as infix operators, so, given two sets a & b, a | b returns  their uniion, a & b computes the instersection, a - b the difference.

## set Literals

The syntax of set literals—{1}, {1, 2}, etc.—looks exactly like the math notation, with one important exception: there’s no literal notation for the empty set, so we must remember to write set()

In [20]:
s ={1}
type(s)

set

In [21]:
s.pop()
type(s)

set

Literal set syntax like {1, 2, 3} is both faster and more readable than calling the constructor.  (e.g., set([1, 2, 3])). The latter form is slower because, to evaluate it,Python has to look up the set name to fetch the constructor, then build a list, and finally pass it to the constructor. 

In [22]:
from dis import dis
dis('{1}')

  1           0 LOAD_CONST               0 (1)
              2 BUILD_SET                1
              4 RETURN_VALUE


In [23]:
dis('set([1])')

  1           0 LOAD_NAME                0 (set)
              2 LOAD_CONST               0 (1)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE


There is no special syntax to represent frozenset literals—they must be created by
calling the constructor.

In [24]:
frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

## Set Comprehensions

In [25]:
from unicodedata import name

{chr(i) for i in range(64, 256) if 'SIGN' in name(chr(i),'')}

{'¢', '£', '¤', '¥', '§', '©', '¬', '®', '°', '±', 'µ', '¶', '×', '÷'}

# dict and set Under the Hood

dicts and sets are much faster at checking for needle in haystack than lists

## Hash Tables in Dictionaries

A hash table is a sparse array. In standard data structure tests, the cells in a hash table are often called "buckets". In a dict hash table, there is a bucket for each item, and it contains two fields: a reference to the key and a reference to the value of the item. Because all buckets have the same size, access to an individual bucket is done by offsey.

Python tries to keep at least 1/3 of the buckets empty; if the hash table become too crowded, it is copied to a new location with room for more buckets.

To put an item in a hash table, the first step is to calculate the hash value of the item key, which is done with the hash() built-in function.

### Hashes and equality

The hash() built-in function works directly with built in types and falls back to calling `__hash__` for user-defined types. If two objects compare equal, their hash values must also be equal, otherwise the hash table algorithm does not work.

Also, to be effective as hash table indexes, hash values should scatter around the index space as much as possible. This means that, ideally, objects that are similar but not equal should have hash values that differ widely.

### The hash table algorithm

To fetch the value at my dict[search_key], python calls has(search_key) to obtain the hash value of search_key and uses the least significant bits of that number as an offset to lookup a bucket in the hash table. Number of bits used depend on the current size of the table. If the bucket is empty, KeyError is raised. Otherwise the found bucket has an item-a found_key:found_value pair-and then Python checks whether search_key == found_key. if they match, thaw was the item sought: found_value is returned.

However, if search_key and found_key do not match, this is a hash collision. This happens because a hash function maps arbitrary objects to a small number of bits, and—in addition—the hash table is indexed with a subset of those bits. In order to resolve the collision, the algorithm then takes different bits in the hash, massages them in a particular way, and uses the result as an offset to look up a different bucket. If that is empty,
KeyError is raised; if not, either the keys match and the item value is returned, or the collision resolution process is repeated

The process to insert or update an item is the same, except that when an empty bucket is located, the new item is put there, and when a bucket with a matching key is found, the value in that bucket is overwritten with the new value. Additionally, when inserting items, Python may determine that the hash table is too crowded and rebuild it to a new location with more room. As the hash table grows, so does the number of hash bits used as bucket offsets, and this keeps the rate of collisions low.

## Practical Consequences of How dict Works

### Keys must be hashable objects

An object is hashable if all of these requirements are met:
1. It supports the hash() function via a hash() method that always returns the same
value over the lifetime of the object.
2. It supports equality via an eq() method.
3. If a == b is True then hash(a) == hash(b) must also be True.

User-defined types are hashable by default because their hash value is their id() and they all compare not equal. 

If you implement a class with a custom `__eq__` method, you must
also implement a suitable `__hash__`, because you must always make
sure that if a == b is True then hash(a) == hash(b) is also True.
Otherwise you are breaking an invariant of the hash table algo‐
rithm, with the grave consequence that dicts and sets will not deal
reliably with your objects. If a custom `__eq__` depends on muta‐
ble state, then `__hash__` must raise TypeError with a message like
unhashable type: 'MyClass'.

### dicts have significant memory overhead

Because a dict uses a hash table internally, and hash tables must be sparse to work, they are not space efficient. For example, if you are handling a large quantity of records, it makes sense to store them in a list of tuples or named tuples instead of using a list of dictionaries in JSON style, with one dict per record. Replacing dicts with tuples reduces the memory usage in two ways: by removing the overhead of one hash table per record and by not storing the field names again with each record.

For user-defined types, the `__slots__` class attribute changes the storage of instance attributes from a dict to a tuple in each instance.

### Key search is very fast

The dict implementation is ana example of trading space for time: dictionaries have significant memory overhead, but they provide fast access regardless of the size of the dictionary-as long as it fits in the memory.

### Key ordering depends on isertion order

When a hash collision happens, the second key ends up in a position that it would not normally occupy if it had been inserted first. So, a dict built as dict([(key1, value1), (key2, value2)]) compares equal to dict([(key2, value2), (key1, value1)]), but their key ordering may not be the same if the hashes of key1 and key2 collide.

### Adding items to a dict may change the order of existing keys

Whenever you add a new item to a dict, the Python interpreter may decide that the hash table of that dictionary needs to grow. This entails building a new, bigger hash table, and adding all current items to the new table. During this process, new (but different) hash collisions may happen, with the result that the keys are likely to be ordered differently in the new hash table. All of this is implementation-dependent, so you cannot reliably predict when it will happen. If you are iterating over the dictionary keys
and changing them at the same time, your loop may not scan all the items as expected, not even the items that were already in the dictionary before you added to it. This is why modifying the contents of a dict while iterating through it is a bad idea. If you need to scan and add items to a dictionary, do it in two steps: read the dict from start to finish and collect the needed additions in a second dict. Then update the first one with it.


## How Sets Work -- Practical Consequences

The set and frozenset types are also implemented with a hash table, except that each bucket holds only a reference to the element (as if it were a key in a dict, but without a value to go with it). In fact, before set was added to the language, we often used dictionaries with dummy values just to perform fast membership tests on the keys.

- Set elements must be hashable objects.
- Sets have a significant memory overhead.
- Membership testing is very efficient.
- Element ordering depends on insertion order.
- Adding elements to a set may change the order of other elements

# Chapter summary

Dictionaries are a keystone of Python. Beyond the basic dict, the standard library offers handy, ready-to-use specialized mappings like defaultdict, OrderedDict, ChainMap, and Counter, all defined in the collections module. The same module also provides the easy-to-extend UserDict class

Two powerful methods available in most mappings are setdefault and update. The setdefault method is used to update items holding mutable values, for example, in a dict of list values, to avoid redundant searches for the same key. The update method allows bulk insertion or overwriting of items from any other mapping, from iterables providing (key, value) pairs and from keyword arguments. Mapping constructors also use update internally, allowing instances to be initialized from mappings, iterables, or keyword arguments

A clever hook in the mapping API is the `__missing__` method, which lets you customize what happens when a key is not found.

The collections.abc module provides the Mapping and MutableMapping abstract base classes for reference and type checking. The little-known MappingProxyType from the types module creates immutable mappings. There are also ABCs for Set and Mutable Set.

The hash table implementation underlying dict and set is extremely fast. Understanding its logic explains why items are apparently unordered and may even be reordered behind our backs. There is a price to pay for all this speed, and the price is in memory